In [1]:
# import pypyodbc
import pandas as pd
from connections import connection
import pyodbc
import calendar

from datetime import datetime as dt
from datetime import timedelta as td
from datetime import date as d

In [2]:
# Using an existing DSN (Data Source Name) for the connection

# ecrs_conn = pypyodbc.connect('DSN=Prototype;UID=ecrs;PWD=St4rs!')
# ecrs_conn = pypyodbc.connect(connection)

ecrs_conn = pyodbc.connect(connection)



In [3]:

# Function to generate a list of dates for a given month and year
def generate_month_dates(year, month):
    # Find the start and end dates of the month
    start_date = dt(year, month, 1)
    
    # Get the last day of the month using calendar.monthrange
    _, last_day = calendar.monthrange(year, month)
    end_date = dt(year, month, last_day)

    # Generate the list of dates
    date_list = []
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime('%Y-%m-%d'))  # Store the date as a string
        current_date += td(days=1)  # Increment by 1 day

    return date_list

# Dictionary to store the dates for each month
all_month_dates = {}

# Loop through each month of 2024
for month in range(1, 13):
    all_month_dates[calendar.month_name[month]] = generate_month_dates(2024, month)

# Print the result (optional)
for month, dates in all_month_dates.items():
    print(f"{month}: {dates}")


January: ['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05', '2024-01-06', '2024-01-07', '2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11', '2024-01-12', '2024-01-13', '2024-01-14', '2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19', '2024-01-20', '2024-01-21', '2024-01-22', '2024-01-23', '2024-01-24', '2024-01-25', '2024-01-26', '2024-01-27', '2024-01-28', '2024-01-29', '2024-01-30', '2024-01-31']
February: ['2024-02-01', '2024-02-02', '2024-02-03', '2024-02-04', '2024-02-05', '2024-02-06', '2024-02-07', '2024-02-08', '2024-02-09', '2024-02-10', '2024-02-11', '2024-02-12', '2024-02-13', '2024-02-14', '2024-02-15', '2024-02-16', '2024-02-17', '2024-02-18', '2024-02-19', '2024-02-20', '2024-02-21', '2024-02-22', '2024-02-23', '2024-02-24', '2024-02-25', '2024-02-26', '2024-02-27', '2024-02-28', '2024-02-29']
March: ['2024-03-01', '2024-03-02', '2024-03-03', '2024-03-04', '2024-03-05', '2024-03-06', '2024-03-07', '2024-03-08', '2024-03-09', '2024

In [9]:
# def Month_df(date_list):
#     # Prepare a list to store DataFrames for each date range
#     df_list = []

#     # Iterate over pairs of consecutive dates using zip
#     for start_date, end_date in zip(date_list[:-1], date_list[1:]):
#         # Build the SQL query
#         ecrs_query = f"""SELECT 
#                             TRN_PK, TRN_StartTime AS StartTime, TRN_EndTime AS EndTime, TRN_ReceiptNumber AS ReceiptNumber, 
#                             CUS_AccountNum AS CustomerAccount, DPT_Name AS Department, CTI_DiscountAmount AS DiscountAmount, 
#                             TRN_Total AS TransactionTotal, TRN_NetSales AS NetSales, 
#                             CASE WHEN CTI_DiscountAmount <= 0 THEN TLI_Quantity ELSE -TLI_Quantity END AS RedemptionCount, 
#                             TLI_Quantity, TLI_Amount
#                         FROM v_TJTrans 
#                         WHERE CMB_Name LIKE '%Senior 10%' 
#                         AND TRN_EndTime BETWEEN '{start_date}' AND '{end_date}' 
#                         AND TRN_AllVoid = 0 
#                         AND ITI_VOID = 0 
#                         AND TRN_Status = 0
#         """
#         # Execute the query and append the result to df_list
#         ecrs_df = pd.read_sql(ecrs_query, ecrs_conn)
#         df_list.append(ecrs_df)
#         # print(f'{start_date} to {end_date} appended')

#         # Handle the last date separately to ensure the full day (up to 23:59:59) is included
#     last_start_date = dt.strptime(date_list[-2], '%Y-%m-%d')
#     last_end_date = dt.strptime(date_list[-1], '%Y-%m-%d')

#     # Adjust the last_end_date to include the full day (23:59:59)
#     last_end_date = last_end_date.replace(hour=23, minute=59, second=59)

#     last_ecrs_query = f"""SELECT 
#                             TRN_PK, TRN_StartTime AS StartTime, TRN_EndTime AS EndTime, TRN_ReceiptNumber AS ReceiptNumber, 
#                             CUS_AccountNum AS CustomerAccount, DPT_Name AS Department, CTI_DiscountAmount AS DiscountAmount, 
#                             TRN_Total AS TransactionTotal, TRN_NetSales AS NetSales, 
#                             CASE WHEN CTI_DiscountAmount <= 0 THEN TLI_Quantity ELSE -TLI_Quantity END AS RedemptionCount, 
#                             TLI_Quantity, TLI_Amount
#                         FROM v_TJTrans 
#                         WHERE CMB_Name LIKE '%Senior 10%' 
#                         AND TRN_EndTime BETWEEN '{last_start_date}' AND '{last_end_date}' 
#                         AND TRN_AllVoid = 0 
#                         AND ITI_VOID = 0 
#                         AND TRN_Status = 0
#     """
#     # Execute the query for the last day and append the result
#     last_ecrs_df = pd.read_sql(last_ecrs_query, ecrs_conn)
#     df_list.append(last_ecrs_df)
#     # print(f'{last_start_date} to {last_end_date} appended')

#     # Concatenate all DataFrames for the month
#     month_df = pd.concat(df_list, ignore_index=True)
#     return month_df


In [4]:
def Month_df(date_list, ecrs_conn):
    # Prepare a list to store DataFrames for each date range
    df_list = []

    # Iterate over pairs of consecutive dates using zip
    for start_date, end_date in zip(date_list[:-1], date_list[1:]):
        # Build the SQL query
        ecrs_query = f"""SELECT 
                            TRN_PK, TRN_StartTime AS StartTime, TRN_EndTime AS EndTime, TRN_ReceiptNumber AS ReceiptNumber, 
                            CUS_AccountNum AS CustomerAccount, DPT_Name AS Department, CTI_DiscountAmount AS DiscountAmount, 
                            TRN_Total AS TransactionTotal, TRN_NetSales AS NetSales, 
                            CASE WHEN CTI_DiscountAmount <= 0 THEN TLI_Quantity ELSE -TLI_Quantity END AS RedemptionCount, 
                            TLI_Quantity, TLI_Amount
                        FROM v_TJTrans 
                        WHERE CMB_Name LIKE '%Senior 10%' 
                        AND TRN_EndTime BETWEEN '{start_date}' AND '{end_date}' 
                        AND TRN_AllVoid = 0 
                        AND ITI_VOID = 0 
                        AND TRN_Status = 0
        """
        try:
            # Execute the query and append the result to df_list
            ecrs_df = pd.read_sql(ecrs_query, ecrs_conn)
            df_list.append(ecrs_df)
            print(f'{start_date} to {end_date} appended')
        except Exception as e:
            print(f"Error executing query for {start_date} to {end_date}: {e}")

    # Handle the last date separately to ensure the full day (up to 23:59:59) is included
    last_start_date = dt.strptime(date_list[-2], '%Y-%m-%d')
    last_end_date = dt.strptime(date_list[-1], '%Y-%m-%d')

    # Adjust the last_end_date to include the full day (23:59:59)
    last_end_date = last_end_date.replace(hour=23, minute=59, second=59)

    last_ecrs_query = f"""SELECT 
                            TRN_PK, TRN_StartTime AS StartTime, TRN_EndTime AS EndTime, TRN_ReceiptNumber AS ReceiptNumber, 
                            CUS_AccountNum AS CustomerAccount, DPT_Name AS Department, CTI_DiscountAmount AS DiscountAmount, 
                            TRN_Total AS TransactionTotal, TRN_NetSales AS NetSales, 
                            CASE WHEN CTI_DiscountAmount <= 0 THEN TLI_Quantity ELSE -TLI_Quantity END AS RedemptionCount, 
                            TLI_Quantity, TLI_Amount
                        FROM v_TJTrans 
                        WHERE CMB_Name LIKE '%Senior 10%' 
                        AND TRN_EndTime BETWEEN '{last_start_date}' AND '{last_end_date}' 
                        AND TRN_AllVoid = 0 
                        AND ITI_VOID = 0 
                        AND TRN_Status = 0
    """
    try:
        # Execute the query for the last day and append the result
        last_ecrs_df = pd.read_sql(last_ecrs_query, ecrs_conn)
        df_list.append(last_ecrs_df)
        print(f'{last_start_date} to {last_end_date} appended')
    except Exception as e:
        print(f"Error executing query for {last_start_date} to {last_end_date}: {e}")

    # Concatenate all DataFrames for the month
    month_df = pd.concat(df_list, ignore_index=True)
    return month_df


In [ ]:
# Define the start and end dates for April 2024
start_date = d(2024, 4, 1)
end_date = d(2024, 4, 30)

# Generate the list of days in the format 'YYYY-MM-DD'
days_in_april_2024 = [(start_date + td(days=i)).strftime('%Y-%m-%d') for i in range((end_date - start_date).days + 1)]

# Print the result
print(days_in_april_2024)


In [ ]:
april_df = Month_df(days_in_april_2024,ecrs_conn)

In [6]:
# Create an empty list to store all DataFrames for each month
all_month_data = []

# Iterate over all months and get the DataFrame for each
for month in all_month_dates:
    month_df = Month_df(all_month_dates[month],ecrs_conn)  # Get the DataFrame for the current month
    month_df.to_csv(f'{month}.csv', index=False)  # Save the DataFrame to a CSV file    
    all_month_data.append(month_df)  # Append to the list
    print(f"Data for {month} appended")

# Concatenate all DataFrames into one
final_df = pd.concat(all_month_data, ignore_index=True)

# Print or return the final DataFrame
print(final_df)

C:\Users\adossantos\AppData\Local\Temp\ipykernel_15624\4211220893.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ecrs_df = pd.read_sql(ecrs_query, ecrs_conn)


2024-01-01 to 2024-01-02 appended


C:\Users\adossantos\AppData\Local\Temp\ipykernel_15624\4211220893.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ecrs_df = pd.read_sql(ecrs_query, ecrs_conn)


2024-01-02 to 2024-01-03 appended
2024-01-03 to 2024-01-04 appended
2024-01-04 to 2024-01-05 appended
2024-01-05 to 2024-01-06 appended
2024-01-06 to 2024-01-07 appended
2024-01-07 to 2024-01-08 appended
2024-01-08 to 2024-01-09 appended
2024-01-09 to 2024-01-10 appended
2024-01-10 to 2024-01-11 appended
2024-01-11 to 2024-01-12 appended
2024-01-12 to 2024-01-13 appended
2024-01-13 to 2024-01-14 appended
2024-01-14 to 2024-01-15 appended
2024-01-15 to 2024-01-16 appended


C:\Users\adossantos\AppData\Local\Temp\ipykernel_15624\4211220893.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ecrs_df = pd.read_sql(ecrs_query, ecrs_conn)


...

2024-12-26 to 2024-12-27 appended
2024-12-27 to 2024-12-28 appended
2024-12-28 to 2024-12-29 appended
2024-12-29 to 2024-12-30 appended
2024-12-30 to 2024-12-31 appended


C:\Users\adossantos\AppData\Local\Temp\ipykernel_15624\4211220893.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  last_ecrs_df = pd.read_sql(last_ecrs_query, ecrs_conn)


2024-12-30 00:00:00 to 2024-12-31 23:59:59 appended


C:\Users\adossantos\AppData\Local\Temp\ipykernel_15624\4211220893.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  month_df = pd.concat(df_list, ignore_index=True)


Data for December appended
          TRN_PK               StartTime                 EndTime  \
0        2492090 2024-01-02 11:36:36.457 2024-01-02 11:37:21.802   
1        2492090 2024-01-02 11:36:36.457 2024-01-02 11:37:21.802   
2        2492090 2024-01-02 11:36:36.457 2024-01-02 11:37:21.802   
3        2492090 2024-01-02 11:36:36.457 2024-01-02 11:37:21.802   
4        2492090 2024-01-02 11:36:36.457 2024-01-02 11:37:21.802   
...          ...                     ...                     ...   
2414315  3251163 2024-12-31 17:41:14.892 2024-12-31 17:42:41.905   
2414316  3251163 2024-12-31 17:41:14.892 2024-12-31 17:42:41.905   
2414317  3251163 2024-12-31 17:41:14.892 2024-12-31 17:42:41.905   
2414318  3251163 2024-12-31 17:41:14.892 2024-12-31 17:42:41.905   
2414319  3251163 2024-12-31 17:41:14.892 2024-12-31 17:42:41.905   

        ReceiptNumber CustomerAccount                      Department  \
0              301353            None  REFRIGERATED                     
1         

In [7]:
final_df.head(15)



,TRN_PK,StartTime,EndTime,ReceiptNumber,CustomerAccount,Department,DiscountAmount,TransactionTotal,NetSales,RedemptionCount,TLI_Quantity,TLI_Amount
0,2492090,2024-01-02 11:36:36.457,2024-01-02 11:37:21.802,301353,None,REFRIGERATED,-0.63,40.63,40.63,1.0,1.0,-0.63
1,2492090,2024-01-02 11:36:36.457,2024-01-02 11:37:21.802,301353,None,REFRIGERATED,-0.30,40.63,40.63,1.0,1.0,-0.30
2,2492090,2024-01-02 11:36:36.457,2024-01-02 11:37:21.802,301353,None,REFRIGERATED,-0.30,40.63,40.63,1.0,1.0,-0.30
3,2492090,2024-01-02 11:36:36.457,2024-01-02 11:37:21.802,301353,None,REFRIGERATED,-0.30,40.63,40.63,1.0,1.0,-0.30
4,2492090,2024-01-02 11:36:36.457,2024-01-02 11:37:21.802,301353,None,REFRIGERATED,-0.30,40.63,40.63,1.0,1.0,-0.30
5,2492090,2024-01-02 11:36:36.457,2024-01-02 11:37:21.802,301353,None,REFRIGERATED,-0.30,40.63,40.63,1.0,1.0,-0.30
6,2492090,2024-01-02 11:36:36.457,2024-01-02 11:37:21.802,301353,None,REFRIGERATED,-0.30,40.63,40.63,1.0,1.0,-0.30
7,2492090,2024-01-02 11:36:36.457,2024-01-02 11:37:21.802,301353,None,REFRIGERATED,-0.30,40.63,40.63,1.0,1.0,-0.30
8,2492090,2024-01-02 11:36:36.457,2024-01-02 11:37:21.802,301353,None,REFRIGERATED,-0.30,40.63,40.63,1.0,1.0,-0.30
9,2492090,2024-01-02 11:36:36.457,2024-01-02 11:37:21.802,301353,None,REFRIGERATED,-0.30,40.63,40.63,1.0,1.0,-0.30


In [ ]:
april_df.head(5)

In [6]:
# month_df.to_csv('january_2024.csv', index= False)

final_df.to_csv('monthly_2024.csv',index= False)

# don't add up all the discounts in sql, I'm missing transactions or discounts, totals are smaller than what Catapult reports